In [ ]:
from my_functions import *
from add_errors import add_errors
from minijpas_LF_and_puricomp import add_errors, nb_or_3fm_cont, compute_L_Lbin_err
import pickle

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67
filter_tags = load_filter_tags()

In [ ]:
dirname = '/home/alberto/almacen/Source_cats'
filename = f'{dirname}/GAL_DR16.npy'
with open(filename, 'rb') as f:
    gal_mock = pickle.load(f)

pm_flx, pm_err = add_errors(gal_mock['pm_flx'], apply_err=True,
                            survey_name='minijpasAEGIS004')

where_bad_flx = ~np.isfinite(pm_flx)
pm_flx[where_bad_flx] = 0.
pm_err[where_bad_flx] = 9999999999.

mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

zspec = gal_mock['z']

N_sources = pm_flx.shape[1]
N_sources

In [ ]:
ew0_cut = 30
ew_other = 100

# Cont est
cont_est_lya, cont_err_lya, cont_est_other, cont_err_other =\
        nb_or_3fm_cont(pm_flx, pm_err, 'nb')

# Lya search
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0_cut, sigma=3)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, cont_est_lya, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    ew_other, obs=True, sigma=5)
other_lines = identify_lines(line_other, cont_est_other, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.16

mag_min = 17
mag_max = 24

nb_min = 1
nb_max = 20

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1
print(f'z interval: ({z_min:0.2f}, {z_max:0.2f})')

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
mag_cut = (mag > mag_min) & (mag < mag_max)

snr = np.empty(N_sources)
for src in range(N_sources):
    l = lya_lines[src]
    snr[src] = pm_flx[l, src] / pm_err[l, src]

nice_lya_mask = (lya_lines >= nb_min) & (lya_lines <= nb_max) & mag_cut & (snr > 6)
nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr, mask=nice_lya_mask
)
print(sum(nice_lya))

In [ ]:
fig = plt.figure()

src = np.random.choice(np.where(nice_lya)[0])
ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

plt.show()

In [ ]:
import os

# Number of sources of each class of QSO for train+test
N_tt_set = sum(nice_lya)
# Select random sources
tt_indices = np.where(nice_lya)[0]

# Save pm_flx, pm_err and lya_lines for the selected sources
savedir = '/home/alberto/almacen/ML_train_sets'
os.makedirs(savedir, exist_ok=True)
np.save(f'{savedir}/pm_flx_tt_gal.npy', pm_flx[:, tt_indices])
np.save(f'{savedir}/pm_err_tt_gal.npy', pm_err[:, tt_indices])
np.save(f'{savedir}/lya_lines_tt_gal.npy', lya_lines[tt_indices])
np.save(f'{savedir}/zspec_tt_gal.npy', zspec[tt_indices])
# Labels: 1 = LAE, 0 = no LAE
labels = np.ones(N_tt_set).astype(int) * 2
np.save(f'{savedir}/labels_gal.npy', labels)